This is the desired YAML-Code from [the issue](https://github.com/cknoll/yamlpyowl/issues/4):

```
- owl_class:
    Class1:
        SubClassOf: "owl:Thing"

- owl_data_property:
    functieCode:
        Domain: Class1
        Range: int 

- owl_class:
    Overeenkomst:
        SubClassOf: (functieCode value 100) or (functieCode value 110)
```

---

This is a proposed implementation in owlready2-based Python:

In [1]:
import owlready2 as owl2
onto = owl2.get_ontology("http://test.org/onto.owl")

with onto:
    class Class1(owl2.Thing):
        """
        - owl_class:
            Class1:
                SubClassOf: "owl:Thing"
        """
        pass

    class functieCode(owl2.DataProperty):
        """
        - owl_data_property:
            functieCode:
                Domain: Class1
                Range: int 
        """
        
        domain = [Class1]
        range = [int]
        
        
    # this is based on https://owlready2.readthedocs.io/en/v0.35/restriction.html
    class Overeenkomst(owl2.Thing):
        """
        - owl_class:
            Overeenkomst:
                SubClassOf: (functieCode value 100) or (functieCode value 110)
        """

        # in owlready2 a SubClassOf statement would use `is_a = ...`
        # this is syntactically correct, but does not enable the reasoner to reclassify
        # an instance based on its data property value.
        # thus, here we choose `equivalent_to` instead of `is_a`
        equivalent_to = [functieCode.value(100) | functieCode.value(110)]

# onto.save("value_restriction_test.xml")

In [2]:
test_instance0 = Class1("t0", functieCode=[100])
test_instance1 = Class1("t1", functieCode=[110])
test_instance2 = Class1("t2", functieCode=[120])

In [3]:
# evaluate the uninferred instances (there should be none)
list(Overeenkomst.instances())

[]

In [4]:
# evaluate the uninferred subclasses (there should be none)
list(Class1.subclasses())

[]

In [5]:
# run the reasoner
owl2.sync_reasoner(infer_property_values=True, debug=False)

In [6]:
# evaluate the inferred instances (there should be two)
list(Class1.instances())

[onto.t0, onto.t1, onto.t2]

In [7]:
# evaluate the inferred subclasses (there should be one)
# because the domain of `functieCode` is Class1, every instance for which that property can be defined,
# must be an instance of Class1.
list(Class1.subclasses())

[onto.Overeenkomst]